In [ ]:
'''
    @Author: King
    @Date: 2019.03.19
    @Purpose: 中文文本分类实战（七）—— Adversarial LSTM模型
    @Introduction:  下面介绍
    @Datasets:cnews 中文文本分类数据集。
    @Link : https://github.com/jiangxinyang227/textClassifier/tree/master/RCNN
    @Reference : https://www.cnblogs.com/jiangxinyang/p/10208290.html
'''

## 数据集

cnews 中文文本分类数据集，其目标是判断一段话的类别，数据预处理如文本分类实战（一）—— word2vec预训练词向量中一样，预处理后的文件为../data/cnews_data/labeledCharTrain.csv。


## Adversarial LSTM模型结构

Adversarial LSTM 模型来源于论文Adversarial Training Methods For Semi-Supervised Text Classification {https://arxiv.org/abs/1605.07725}。其模型结构如下右图所示：

![avatar](../images/Adversarial_LSTM.png)

　上图中左边为正常的LSTM结构，右图为Adversarial LSTM结构，可以看出在输出时加上了噪声。

　　Adversarial LSTM的<span class="burk">核心思想是</span>通过<span class="girk">对word Embedding上添加噪音生成对抗样本，将对抗样本以和原始样本 同样的形式喂给模型，得到一个Adversarial Loss，通过和原始样本的loss相加得到新的损失，通过优化该新 的损失来训练模型</span>，作者认为这种方法能对word embedding加上正则化，避免过拟合。

In [3]:
import os
import csv
import time
import datetime
import random
import json
import threading

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

d:\progrom\python\python\python3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# 配置参数

class TrainingConfig(object):
    epoches = 2
    evaluateEvery = 10
    checkpointEvery = 10
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = 128  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    epsilon = 5
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/cnews_data/labeledCharTrain.csv"                               # update
    
    stopWordSource = "../data/cnews_data/stopwords.txt"
    
    numClasses = 10                                                                     # update
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

## 生成训练数据

　　1）将数据加载进来，将句子分割成词表示，并去除低频词和停用词。

　　2）将词映射成索引表示，构建词汇-索引映射表，并保存成json的数据格式，之后做inference时可以用到。（注意，有的词可能不在word2vec的预训练词向量中，这种词直接用UNK表示）

　　3）从预训练的词向量模型中读取出词向量，作为初始化值输入到模型中。

　　4）将数据集分割成训练集和测试集

In [18]:
# 整块 update
import sys
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self.numClasses = config.numClasses        # update
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self.indexFreqs = []  # 统计词空间中的词在出现在多少个review中
        
        self._wordToIndex = {}
        self._indexToWord = {}
        self.label_to_index = {}
        self.index_to_label = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["class"].tolist()                # update
        review = df["review_cut_word"].tolist()
        reviews = [str(line).strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _labelProcess(self, y, label_to_index):
        """
        将数据集中的每个label用index表示
        label_to_index“pad”对应的index为0
        """
        input_y = []
        for item in y:
            temp = [0] * self.numClasses
            temp[label_to_index[item]] = 1
            input_y.append(temp)
        
        return input_y
    
    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        input_y = self._labelProcess(y,self.label_to_index)
#         print("input_y:{0}".format(input_y[0:2]))
        
#         sys.exit(0)
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append(input_y[i])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews,labels):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 得到逆词频
        self._getWordIndexFreq(vocab, reviews)

        labels = set(labels)

        for i, label in enumerate(labels):
            self.label_to_index[label] = i
            self.index_to_label[int(i)] = label
        
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex_cnews.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord_cnews.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
        with open('../data/wordJson/label_to_index_cnews.json', 'w', encoding='utf-8') as f:
            json.dump(self.label_to_index, f, ensure_ascii=False)

        with open('../data/wordJson/index_to_label_cnews.json', 'w', encoding='utf-8') as f:
            json.dump(self.index_to_label, f, ensure_ascii=False)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../data/ChnSentiCorp_cn_data/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                #print(word + "不存在于词向量中")
                pass
                
        return vocab, np.array(wordEmbedding)
    
    def _getWordIndexFreq(self, vocab, reviews):
        """
        统计词汇空间中各个词出现在多少个文本中
        """
        reviewDicts = [dict(zip(review, range(len(review)))) for review in reviews]
        indexFreqs = [0] * len(vocab)
        for word in vocab:
            count = 0
            for review in reviewDicts:
                if word in review:
                    count += 1
            indexFreqs[self._wordToIndex[word]] = count
        
        self.indexFreqs = indexFreqs
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r", encoding="utf-8") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews,labels)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        print("self.trainReviews[0:2]:{0}".format(self.trainReviews[0:3]))
        print("self.trainLabels[0:2]:{0}".format(self.trainLabels[0:3]))
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        print("self.evalReviews[0:2]:{0}".format(self.evalReviews[0:3]))
        print("self.evalLabels[0:2]:{0}".format(self.evalLabels[0:3]))
        
        
data = Dataset(config)
data.dataGen()

self.trainReviews[0:2]:[[   1    1    1    1    1    1    1    1    1    1    1    1 1131    1
     1    1    1  478  142    1   27    2    1  483    1    1    1    1
     1    1 1281    1    1    1    1    1    1  886    1    1    1    1
     1    1    1    1    1    1 1005    1    1    1   13    1    1    1
     1    1    1    1    1    1    1  203  215    1    1    1    1    1
     1    1   27    1    1    1    1    1  102    1  152    1    1    1
     1  451    1    1  753    1    1  226    1    1   65    1    1    1
     1    1    1   19    1    1    1    1    1  849    1    1 2243    1
     1    1    1    1    1    1    1    1  878    1    1    1    1    1
     1    1    1    1    1    1    1    1    1    1    1  419    1    1
   196    1    1    1    1    1    1    1    1    1    1    1    1    1
     1    1   10    1    1    8    1    1 1654    1    1    1    1    1
     1    1    1   36    1    1    1 1151    1    1    1    1    1    1
     1    1  120    1    1 1162    1    1

In [19]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (3999, 200)
train label shape: (3999, 10)
eval data shape: (1000, 200)


In [20]:
print(data.wordEmbedding.shape)

(3302, 200)


## 生成batch数据集

　　采用生成器的形式向模型输入batch数据集，（生成器可以避免将所有的数据加入到内存中）

In [21]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

## Adversarial LSTM模型

In [22]:
# 构建模型
class AdversarialLSTM(object):
    """
    Adversarial LSTM模型 用于文本分类
    """
    def __init__(self, config, wordEmbedding, indexFreqs):

        # 定义模型的输入
        self.numClasses = config.numClasses      # update
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, self.numClasses], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.config = config
        
        # 根据词的频率计算权重
        indexFreqs[0], indexFreqs[1] = 20000, 10000
        weights = tf.cast(tf.reshape(indexFreqs / tf.reduce_sum(indexFreqs), [1, len(indexFreqs)]), dtype=tf.float32)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用词频计算新的词嵌入矩阵
            normWordEmbedding = self._normalize(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec"), weights)
            
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(normWordEmbedding, self.inputX)
            
         # 计算二元交叉熵损失 
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.scores,self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                #self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.inputY)
                loss = tf.reduce_mean(losses)
        
        with tf.name_scope("perturLoss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                perturScores,perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                perturLosses = tf.nn.softmax_cross_entropy_with_logits(logits=perturScores, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)
                
        self.loss = loss + perturLoss
        
        # accuracy
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.inputY, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
            
    def _Bi_LSTMAttention(self, embeddedWords):
        """
        Bi-LSTM + Attention 的模型结构
        """
        
        config = self.config
        
        # 定义双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
           
            # 定义前向LSTM结构
            lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, state_is_tuple=True),
                                                         output_keep_prob=self.dropoutKeepProb)
            # 定义反向LSTM结构
            lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, state_is_tuple=True),
                                                         output_keep_prob=self.dropoutKeepProb)


            # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
            # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                          self.embeddedWords, dtype=tf.float32,
                                                                          scope="bi-lstm")

        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self._attention(H)
            outputSize = config.model.hiddenSizes
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, self.numClasses],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[self.numClasses]), name="outputB")
            self.scores = tf.nn.xw_plus_b(output, outputW, outputB, name='scores')
            self.predictions = tf.argmax(self.scores, 1, name='predictions')
            
        return self.scores,self.predictions
    
    def _attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output
    
    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        
        mean = tf.matmul(weights, wordEmbedding)
        print(mean)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)
        
        var = tf.matmul(weights, powWordEmbedding)
        print(var)
        stddev = tf.sqrt(1e-6 + var)
        
        return (wordEmbedding - mean) / stddev
    
    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, self.config.model.epsilon)
        return embedded + perturb
    
    def _scaleL2(self, x, norm_length):
        # shape(x) = (batch, num_timesteps, d)
        # Divide x by max(abs(x)) for a numerically stable L2 norm.
        # 2norm(x) = a * 2norm(x/a)
        # Scale over the full sequence, dims (1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keepdims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(
            tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keepdims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [23]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = 0
    try:
        auc = roc_auc_score(trueY, predY)
    except ValueError:
        pass
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型
import os
import shutil
# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding
indexFreqs = data.indexFreqs

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = AdversarialLSTM(config, wordEmbedding, indexFreqs)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        acc_summary = tf.summary.scalar('accuracy', lstm.accuracy)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        shutil.rmtree('../model/AdversarialLSTM/savedModel')  # 去除 model 目录下文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/AdversarialLSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.accuracy],
                feed_dict=feed_dict
            )
            timeStr = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            trainSummaryWriter.add_summary(summaries, step)

        def devStep(batchX, batchY, writer=None):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.accuracy],
                feed_dict=feed_dict
            )
            timeStr = datetime.datetime.now().isoformat()
            print("Evaluation:{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        devStep(evalReviews, evalLabels, writer=evalSummaryWriter)
                        
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/AdversarialLSTM/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

Tensor("embedding/MatMul:0", shape=(1, 200), dtype=float32)
Tensor("embedding/MatMul_1:0", shape=(1, 200), dtype=float32)
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/kernel:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/bias:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/bias:0/grad/sparsity is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/bw/lstm_cell/kernel:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/bw/lstm_cell/kernel:0/grad/sparsity is illegal; using Bi-LSTM/bi-

Evaluation:2019-04-19T10:53:17.617130: step 73, loss 0.705468, acc 0.972
Evaluation:2019-04-19T10:53:38.918165: step 74, loss 0.693587, acc 0.973
Evaluation:2019-04-19T10:53:59.451257: step 75, loss 0.671539, acc 0.972
Evaluation:2019-04-19T10:54:21.021573: step 76, loss 0.642029, acc 0.973
Evaluation:2019-04-19T10:54:42.162039: step 77, loss 0.607703, acc 0.973
Saved model checkpoint to ../model/AdversarialLSTM/model/my-model-70

2019-04-19T10:54:48.847162: step 78, loss 5.50112, acc 0.0234375
2019-04-19T10:54:54.489074: step 79, loss 5.21653, acc 0.0390625
2019-04-19T10:54:59.773943: step 80, loss 5.15896, acc 0.046875

Evaluation:
Evaluation:2019-04-19T10:55:20.840606: step 81, loss 0.598361, acc 0.979
Evaluation:2019-04-19T10:55:42.222428: step 82, loss 0.621716, acc 0.979
Evaluation:2019-04-19T10:56:04.181705: step 83, loss 0.632874, acc 0.978
Evaluation:2019-04-19T10:56:26.229743: step 84, loss 0.62964, acc 0.978
Evaluation:2019-04-19T10:56:48.141147: step 85, loss 0.612801, acc 